## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = '/datasets/data.csv'
clents_table = pd.read_csv(path)

In [3]:
print(clents_table.head(10))
print(clents_table.info())
print(clents_table['children'].value_counts())
print(clents_table['family_status_id'].value_counts())
print(clents_table['debt'].value_counts())
print('======================================')
print(clents_table['purpose'].unique())
display(clents_table['education'].value_counts())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

### Вывод

В столбцах days_employed и total_income есть пропущенные значения. Пропуски могут быть связаны с трудностью достоверного описания своего стажа и нежеланием указывать реальный доход заемщика. Также total_income имеет вещественный тип, и нужно будет преобразовать его к целочичленному. Есть много вариантов заполнения колонки цели кредита, будет необходимо лемматизировать и привезти к каким-то основным видам. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
#print(clents_table['total_income'].isnull().count())
print('===============')
clents_table['education'] = clents_table['education'].str.lower()
# найдем среднее значение дохода и введем коэффициенты на влияние на доход по образованию и типу занятости, 
# чтобы в дальнейшем заменить пропущенные значения дохода в зависимости от значений коэффициентов 
mean_income = clents_table['total_income'].mean()
data_income_education_mean = pd.DataFrame(clents_table.groupby('education')['total_income'].mean())
data_income_education_mean['k_education'] = data_income_education_mean['total_income']/mean_income
data_income_income_type_mean = pd.DataFrame(clents_table.groupby('income_type')['total_income'].mean())
data_income_income_type_mean['k_income_type'] = data_income_income_type_mean['total_income']/mean_income
# теперь соединим получившиеся коэффициенты и перемножим для учета совместного влияния
clents_table = clents_table.merge(data_income_income_type_mean['k_income_type'], how = 'left', on = 'income_type')
clents_table = clents_table.merge(data_income_education_mean['k_education'], on='education', how='left')
clents_table['teoretic_income']=clents_table['k_education']*clents_table['k_income_type']*mean_income
clents_table.loc[clents_table['total_income'].isnull(),'total_income'] = clents_table['teoretic_income']

clents_table.loc[clents_table['days_employed'].isnull(),'days_employed'] = 0
print(clents_table.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 15 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
k_income_type       21525 non-null float64
k_education         21525 non-null float64
teoretic_income     21525 non-null float64
dtypes: float64(5), int64(5), object(5)
memory usage: 3.3+ MB
None


### Вывод  

Пропущенных значений в колонках children и family_status_id не обнаружено. Пропущенные значения дохода заменены на усредненные в зависимости от уровня образования и должности. С колонки days_employed работать не планирую, поэтому заменю пропущенные значения на 0

### Замена типа данных

In [5]:
clents_table['total_income'] = pd.to_numeric(round(clents_table['total_income'],0),downcast='integer')
print(clents_table.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 15 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int32
purpose             21525 non-null object
k_income_type       21525 non-null float64
k_education         21525 non-null float64
teoretic_income     21525 non-null float64
dtypes: float64(4), int32(1), int64(5), object(5)
memory usage: 3.2+ MB
None


### Вывод

In [6]:
#Значения дохода переведены в целочисленный тип, предварительно округлены.

### Обработка дубликатов

In [7]:
print(clents_table['education'].unique())

data_count = pd.DataFrame([]) 
data_count['children'] = clents_table['children'].value_counts()
clents_table.loc[clents_table['children']==20,'children'] = 2
clents_table.loc[clents_table['children']==-1,'children'] = 1
print(data_count)
print('===============')

#clents_table.loc[clents_table['education']==2,'education'] = 'неоконченное высшее'
#clents_table.loc[clents_table['education']==1,'education'] = 'среднее'
print('===============')
print(clents_table['education'].unique())

print(clents_table.shape)
clents_table = clents_table.drop_duplicates().reset_index(drop=True)
print(clents_table.shape)

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
     children
 0      14149
 1       4818
 2       2055
 3        330
 20        76
-1         47
 4         41
 5          9
['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
(21525, 15)
(21454, 15)


### Вывод

Встречаются значения количества детей равные -1 и 20, являющиеся, скорее всего, человеческими ошибками, и означающими количество равное 1 и 2. Сделал замену значений исходя из данного предположения. 
До удаления дубликатов было 21525 строк в таблицу. Удалился 71 дубликат.

In [8]:
# Пример
display(clents_table['education_id'].value_counts())
display(clents_table['education'].value_counts())

1    15172
0     5250
2      744
3      282
4        6
Name: education_id, dtype: int64

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

### Лемматизация

In [9]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
#сначала выявим все использованные леммы
lemmas_data = clents_table[['purpose']].drop_duplicates().reset_index(drop=True)

def lemmatize_data(row):
    lemmas = m.lemmatize(row)
    return lemmas

lemmas_data['purpose'] = lemmas_data['purpose'].apply(lemmatize_data)
# лемматизируем, исходя из одинакового значения лем Образование и Высокий, а также Недвижимость и Жилье
def lemmatize(row):
    lemmas = ''.join(m.lemmatize(row))
    if 'образование' in lemmas or 'высокий' in lemmas :
        return 'образование'
    if 'ремонт' in lemmas:
        return 'ремонт'
    if 'недвижимость' in lemmas or 'жилье' in lemmas:
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'свадьба' in lemmas:
        return 'свадьба'
    return lemmas

clents_table['purpose_lemmas'] = clents_table['purpose'].apply(lemmatize)
print(clents_table['purpose_lemmas'].value_counts())

недвижимость    10204
автомобиль       4306
образование      4013
свадьба          2324
ремонт            607
Name: purpose_lemmas, dtype: int64


### Вывод

Видно, что все причины можно свести к следующим 5: образование(высокий), ремонт жилья, покупка недвижимости(жилье), покупка авто, свадьба.

### Категоризация данных

In [33]:
income_percentile = np.percentile(clents_table['total_income'], [0, 25, 50, 75])

clents_table['income_level'] = 1
for i in range(4):
    clents_table['income_level'] = clents_table['income_level'] + (clents_table['total_income'] < income_percentile[i-1])

print(clents_table['income_level'].value_counts())
display(clents_table.head(10))

display(clents_table['education_id'].value_counts())
display(clents_table['education'].value_counts())

education_table = pd.DataFrame(clents_table[['education','education_id']])
display(education_table)

education_table = education_table.drop_duplicates().reset_index(drop=True)
print(education_table)


1    5586
2    5528
4    5364
3    4976
Name: income_level, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,k_income_type,k_education,teoretic_income,purpose_lemmas,income_level
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,0.963911,1.237246,199667.025380,недвижимость,1
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,0.963911,0.918131,148168.256785,автомобиль,3
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,0.963911,0.918131,148168.256785,недвижимость,3
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,0.963911,0.918131,148168.256785,образование,1
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0.819051,0.918131,125901.008508,свадьба,2
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255764,покупка жилья,1.209023,1.237246,250440.123797,недвижимость,1
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240526,операции с жильем,1.209023,1.237246,250440.123797,недвижимость,1
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135824,образование,0.963911,0.918131,148168.256785,образование,3
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95857,на проведение свадьбы,0.963911,1.237246,199667.025380,свадьба,4
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144426,покупка жилья для семьи,0.963911,0.918131,148168.256785,недвижимость,3


1    15172
0     5250
2      744
3      282
4        6
Name: education_id, dtype: int64

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
...,...,...
21449,среднее,1
21450,среднее,1
21451,среднее,1
21452,среднее,1


             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


### Вывод

Поэкспериментировав со значениями процентилей, присвоил 4 значения уровня дохода, исходя из схожих по размерам групп. 1 - самый высокий доход, 4 - самый низкий
Столбец education_id по сути дублирует значение столбца education, заменяя каждую категорию на определенную цифру. Дубликаты из столбцов были удалены ранее. Не совсем понятно, какую еще задачу необходимо решить.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [11]:
def debt_converion(factor):
    data_factor = pd.pivot_table(clents_table,index=[factor], columns = clents_table['debt'], values=['gender'], 
                             aggfunc='count')
    data_factor.columns = ['0','1']
    data_factor.loc[data_factor['1'].isnull()] = 0
    data_factor.loc[data_factor['0'].isnull()] = 0
    data_factor['sum'] = data_factor['0'] +data_factor['1']
    data_factor['convers'] = 100 - round(data_factor['0']/data_factor['sum']*100,2)
    data_factor.loc[data_factor['convers'].isnull()] = 0
    print(data_factor)
    
debt_converion('children')
    

                0       1      sum  convers
children                                   
0         13028.0  1063.0  14091.0     7.54
1          4410.0   445.0   4855.0     9.17
2          1926.0   202.0   2128.0     9.49
3           303.0    27.0    330.0     8.18
4            37.0     4.0     41.0     9.76
5             0.0     0.0      0.0     0.00


### Вывод

От изменения количества детей шанс невозврата кредита заметно не меняется, но незначительно у заемщиков без детей он ниже - всего 7,5% вместо 8-10% у остальных категорий.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [12]:
debt_converion('family_status')

                           0    1    sum  convers
family_status                                    
Не женат / не замужем   2536  274   2810     9.75
в разводе               1110   85   1195     7.11
вдовец / вдова           896   63    959     6.57
гражданский брак        3763  388   4151     9.35
женат / замужем        11408  931  12339     7.55


### Вывод

Менее всего рисков с невозвратом связано с заемщиками, состоящими сейчас или ранее в браке. Особенно, если супруг/супруга скончались. При этом имущество по наследству перешло к супругу.
Самыми рискованными заемщиками являются холостые и незамужние. В случае потери места работы у них нет поддержки со стороны супруга.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [13]:
debt_converion('income_level')

                 0    1   sum  convers
income_level                          
1             5207  379  5586     6.78
2             5028  500  5528     9.04
3             4541  435  4976     8.74
4             4937  427  5364     7.96


### Вывод

Меньше всего проблем с выплатой кредитов у самых обеспеченных заемщиков. Среди остальных заметного влияния доход не оказывает.

- Как разные цели кредита влияют на его возврат в срок?

In [14]:
debt_converion('purpose_lemmas')

                   0    1    sum  convers
purpose_lemmas                           
автомобиль      3903  403   4306     9.36
недвижимость    9457  747  10204     7.32
образование     3643  370   4013     9.22
ремонт           572   35    607     5.77
свадьба         2138  186   2324     8.00


### Вывод

Больше всего просрочек среди кредитов на образование и покупку автомобиля, что может быть связано также с возрастной категорией, которая пользуется данными кредитами. Меньше всего просрочек по кредитам на ремонт жилья, возможно связано с меньшей суммой кредитов. 

### Шаг 4. Общий вывод

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

Меньше всего рисков связано с заемщиками, не имеющими детей, но находящимися или бывшими ранее в браке, имеющих высокий уровень дохода, берущих кредит на покупку или ремонт недвижимости либо на свадьбу.
Наиболее рискованные холостые/незамужние заемщики, имеющие 4 детей, берут кредит для оплаты образования или покупки автомобиля, имеющие средний уровень дохода.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.